<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Supmodel_v2/M_maxlen7_350_icd/CNNmodelv2_v1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Builf train and evaluate
# Save model for offline scoring on test
# Model name: CCP_ICD10_Searchengine/CNNmodel_v1.0.ipynb
# References
#https://github.com/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments/Model-2-keras_lstmConv.ipynb
#https://github.com/njainds/NLP/blob/master/kaggle/QIQC/22nd%20Place%20Solution%20-%206%20models%20%2B%20POS%20tagging.ipynb
#https://github.com/Cheneng/DPCNN/blob/master/model/DPCNN.py


In [0]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [1]:

import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')

import os
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam, RMSprop
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import callbacks

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2"
os.chdir("/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/d_maxlen7_350_icd")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
x_train = np.load("train_X.npy")
x_test = np.load("test_X.npy")
y_train = np.load("train_y.npy")
y_test = np.load("test_y.npy")
features = np.load("test_y.npy")
icd_dict = np.load("icd_dict.npy").item()
word_index = np.load("word_index.npy").item()
embeddings = np.load("embedding_matrix.npy")

In [0]:
class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=4590., mode='triangular',gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
    
        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size,
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        K.set_value(self.model.optimizer.lr, self.clr())

In [12]:
class RocAucEvaluation(keras.callbacks.Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

#del check_point, ra_val, early_stop
!rm ./best_model.hdf5
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(x_test, y_test), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

rm: cannot remove './best_model.hdf5': No such file or directory


In [8]:
#configs
os.chdir("/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/M_maxlen7_350_icd")
max_features = embeddings.shape[0]
maxlen = x_train.shape[1]
embed_size = embeddings.shape[1]
n_class = y_train.shape[1]
print(max_features,maxlen, embed_size, n_class)

13735 7 200 350


In [0]:
def get_model():
    keras.backend.clear_session()
    inp = Input(shape=(maxlen,))
    x   = Embedding(max_features, embed_size, weights=[embeddings], trainable=True)(inp)
    x   = SpatialDropout1D(rate=0.2)(x)
    x1  = Conv1D(128, kernel_size=1, padding='valid', kernel_initializer='he_uniform')(x)
    x2  = Conv1D(128, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x)
    x3  = Conv1D(128, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x)
    x4  = Conv1D(128, kernel_size=4, padding='valid', kernel_initializer='he_uniform')(x)
    x5  = Conv1D(128, kernel_size=5, padding='valid', kernel_initializer='he_uniform')(x)
    
    x1  = GlobalMaxPooling1D()(x1)
    x2  = GlobalMaxPooling1D()(x2)
    x3  = GlobalMaxPooling1D()(x3)
    x4  = GlobalMaxPooling1D()(x4)
    x5  = GlobalMaxPooling1D()(x5)
    c   = concatenate([x1,x2,x3,x4,x5])
    y   = Dense(512, activation='relu')(c)
    y   = Dropout(0.2)(y)
    out = Dense(n_class, activation='softmax')(y)
    model=Model(inputs=inp,outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=["accuracy"])
    return model
#print(model.summary())

In [10]:
x_train.shape

(52033, 7)

In [13]:
# Kfold training of model

kfold = StratifiedKFold(n_splits=8, random_state=2019, shuffle=True)
pred_test = np.zeros((x_test.shape[0], n_class))

for i, (train_index, valid_index) in enumerate(kfold.split(x_train, np.argmax(y_train, axis=1))):
    X_train, X_val, Y_train, Y_val = x_train[train_index], x_train[valid_index], y_train[train_index], y_train[valid_index]
    
    file_path = "best_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
    ra_val = RocAucEvaluation(validation_data=(X_val, Y_val), interval = 1)
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
    
    model = get_model()
    
    print("### Model with seed: {}  for fold no. {} ####".format( 2019, i))
    
    clr = CyclicLR(base_lr=1e-6, max_lr=5e-4,step_size=10840., mode='exp_range',gamma=0.99994)
    model.fit(X_train, Y_train, batch_size = 24, epochs = 10, validation_data = (X_val, Y_val), verbose = 1, callbacks = [clr, ra_val, check_point, early_stop])
    
    pred_train = model.predict([X_val], batch_size=24, verbose=2)
    pred_test += model.predict([x_test], batch_size=16, verbose=1)/8
    os.remove(file_path)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
### Model with seed: 2019  for fold no. 0 ####
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 45382 samples, validate on 6651 samples
Epoch 1/10
45382/45382 [==============================] - 32s 710us/step - loss: 4.9219 - acc: 0.1326 - val_loss: 3.6684 - val_acc: 0.3087

 ROC-AUC - epoch: 1 - score: 0.901586

Epoch 00001: val_loss improved from inf to 3.66841, saving model to best_model.hdf5
Epoch 2/10
45382/45382 [==============================] - 29s 631us/step - loss: 2.8979 - acc: 0.4103 - val_loss: 2.1510 - val_acc: 0.5384

 ROC-AUC - epoch: 2 - score: 0.971983

Epoch 00002: val_loss improved from 3.66841 to 2.15102, saving model to best_model.hdf5
Epoch 3/10
45382/45382 [==============================] - 29s

In [0]:
# ROC scores on Test set for different codes
scores=[]
for i in range(396):
  score = roc_auc_score(y_test[:,i], pred_test[:,i])
  scores.append(score)
  #print("score for index %s is %d" (i,score))

low = list(np.argsort(scores)[:5])
high = list(np.argsort(scores)[::-1][:250])
print('high scores is {}'.format(np.mean([scores[i] for i in high])))
print('low scores is {}'.format(np.mean([scores[i] for i in low])))
# ROC is not a problem in multi-class prediction

high scores is 0.9976635485766142
low scores is 0.8496196545022776


In [0]:
# Saving predictions for error analysis
os.mkdir('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/M_maxlen7_350_icd/run1_kfold_73.4/')
os.chdir('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata_v2/M_maxlen7_350_icd/run1_kfold_73.4')

itoicd = dict((v,k) for k,v in icd_dict.items())
test_pred = np.argmax(pred_test, axis=1)
test_act = np.argmax(y_test, axis=1) 
mdf = pd.DataFrame({'test_act':[itoicd[i] for i in list(test_act)],'test_pred':[itoicd[i] for i in list(test_pred)]})
mdf.to_csv('pred_test.csv',index=None)

In [0]:
# Top Codes with low accuracy on test set
#pred_test
acc = [len(mdf[(mdf['test_pred']==i) & (mdf['test_act']==i)])/len(mdf[mdf['test_act']==i]) for i in mdf['test_act'].values]
mdf2 = pd.DataFrame({'class_index': mdf['test_act'].values,'accuracy':acc})
#print(len(mdf2[mdf2['accuracy']<0.5].class_index.unique().tolist()))
codes= mdf2[mdf2['accuracy']<0.5].class_index.unique().tolist()
mdf2.to_csv('error_codes.csv', index=None)

In [17]:
# Accuracy on various condifence levels and sample coverage on Test set
test_pred = np.argmax(pred_test, axis=1)
test_pred_proba = np.max(pred_test,axis=1)
mdf3 = pd.DataFrame({'test_act':list(test_act),'test_pred':list(test_pred), 'prob': list(test_pred_proba)})
#del CUTOFF
for CUTOFF in range(0,100,5):
    acc = len(mdf3[(mdf3['prob']>CUTOFF/100) & (mdf3['test_act']==mdf3['test_pred'])])/len(mdf3[(mdf3['prob']>CUTOFF/100)])
    cov = len(mdf3[(mdf3['prob']>CUTOFF/100)])/len(mdf3)
    print("Confidence: %f ## Accuracy: %f ## Coverage on Test dataset: %f" % (CUTOFF/100,acc,cov))
    #print("coverage with this accuracy level on Test dataset is {}".format(cov))


  

Confidence: 0.000000 ## Accuracy: 0.734694 ## Coverage on Test dataset: 1.000000
Confidence: 0.050000 ## Accuracy: 0.748106 ## Coverage on Test dataset: 0.981840
Confidence: 0.100000 ## Accuracy: 0.754626 ## Coverage on Test dataset: 0.971982
Confidence: 0.150000 ## Accuracy: 0.764270 ## Coverage on Test dataset: 0.957454
Confidence: 0.200000 ## Accuracy: 0.773353 ## Coverage on Test dataset: 0.942407
Confidence: 0.250000 ## Accuracy: 0.783007 ## Coverage on Test dataset: 0.926150
Confidence: 0.300000 ## Accuracy: 0.793064 ## Coverage on Test dataset: 0.907644
Confidence: 0.350000 ## Accuracy: 0.806856 ## Coverage on Test dataset: 0.882912
Confidence: 0.400000 ## Accuracy: 0.820198 ## Coverage on Test dataset: 0.858008
Confidence: 0.450000 ## Accuracy: 0.837059 ## Coverage on Test dataset: 0.827914
Confidence: 0.500000 ## Accuracy: 0.849130 ## Coverage on Test dataset: 0.795572
Confidence: 0.550000 ## Accuracy: 0.862754 ## Coverage on Test dataset: 0.766171
Confidence: 0.600000 ## Accu

In [21]:
# Steps- for re generating the Data for modeling

#datasize for icd codes

strain = np.argmax(np.vstack([y_train,y_test]), axis=1)
codesize = [sum(strain==icd_dict[i]) for i in codes]
#print(len([i for i in codesize if i < 30]))
codesize2 = [sum(strain==icd_dict[i]) for i in (set([itoicd[i] for i in set(strain)])-set(codes))]
#print(len([i for i in codesize2 if i < 30]))

print("mean dataset size of error codes is %d" % (np.mean(codesize)))
print("mean  dataset size of non-error codes is %d" % (np.mean(codesize2)))
print("median dataset size of error codes is %d" % (np.median(codesize)))
print("median  dataset size of non-error codes is %d" % (np.median(codesize2)))

# word entropy for icd codes

idx = [[j for j in range(len(strain)) if strain[j]==icd_dict[i]] for i in codes]
idx2 = [[j for j in range(len(strain)) if strain[j]==icd_dict[i]] for i in (set([itoicd[i] for i in set(strain)])-set(codes))]
x = np.vstack([x_train, x_test])
word_ent = [len(set([j for i in [[list(x[i]) for i in p] for p in idx][p] for j in i])) for p in range(len([[list(x[i]) for i in p] for p in idx]))]
word_ent2 = [len(set([j for i in [[list(x[i]) for i in p] for p in idx2][p] for j in i])) for p in range(len([[list(x[i]) for i in p] for p in idx2]))]

print("#################################################")
print("mean # of unqiue words in error codes is %d" % (np.mean(word_ent)))
print("mean # of unqiue words in non-error codes is %d" % (np.mean(word_ent2)))
print("median # of unqiue words in error codes is %d" % (np.median(word_ent)))
print("median # of unqiue words in non-error codes is %d" % (np.median(word_ent2)))


#codesize


mean dataset size of error codes is 70
mean  dataset size of non-error codes is 193
median dataset size of error codes is 50
median  dataset size of non-error codes is 104
#################################################
mean # of unqiue words in error codes is 76
mean # of unqiue words in non-error codes is 109
median # of unqiue words in error codes is 58
median # of unqiue words in non-error codes is 80


In [0]:
# Saving Model
# serialize model to JSON
model_json = model.to_json()
with open("model_v1.0.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_v1.0.h5")
print("Saved model to disk")
 
# later...

Saved model to disk


In [0]:
# load json and create model
json_file = open('model_v1.0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_v1.0.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3), metrics=["accuracy"])
score = np.argmax(loaded_model.predict(DATASET, batch_size = 16, verbose = 1), axis=1)